In [1]:
# --- AGENTE Q&A CONDOMÍNIO SOLAR COM LANGGRAPH E MEMÓRIA ---
# COM TRUNCAMENTO AUTOMÁTICO DE CONTEXTO
# Autor: William Lapa

import operator
import os
import json
from typing import TypedDict, Annotated, List, Dict, Any, Optional
from dataclasses import dataclass
from datetime import datetime
import io
import tiktoken
import numpy as np

# LangChain imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_groq import ChatGroq

# OCR imports
import fitz  # PyMuPDF
import pytesseract
from PIL import Image

# Display imports
from IPython.display import display, Markdown

# --- CONFIGURAÇÕES GLOBAIS ---
CACHE_DIR = "processed_docs_cache"
DOCS_DIR = "docs_condominio"


@dataclass
class QASession:
    """Classe para gerenciar sessão de Q&A com memória"""
    conversation_history: List[Dict[str, str]]
    retriever: Optional[Any]
    retriever_initialized: bool
    session_id: str
    documents_loaded: bool = False
    
    def __post_init__(self):
        if not self.conversation_history:
            self.conversation_history = []
        if not self.session_id:
            self.session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# --- ESTADO DO GRAFO ---
class AgentState(TypedDict):
    """Estado do agente Q&A para condomínios"""
    question: str
    documents: Annotated[List[Document], operator.add]
    answer: str
    retriever: object
    retriever_initialized: bool
    conversation_history: List[Dict[str, str]]
    session_context: Optional[QASession]


class SolarCondominiumQA:
    """Agente Q&A especializado para condomínio Solar com OCR e cache"""

    
    def __init__(self, docs_directory: str = DOCS_DIR, provider: str = "openai"):
        self.docs_directory = docs_directory
        self.cache_dir = CACHE_DIR
        self.provider = provider
        self.session = QASession(
            conversation_history=[],
            retriever=None,
            retriever_initialized=False,
            session_id=""        
        )
        
        
        # Inicializar componentes
        self._setup_directories()
        self._initialize_llm_dynamic(self.provider)  # Escolha o LLM desejado
        self._build_graph()

    def _count_tokens(self, text: str) -> int:
        """Conta o número de tokens usando o codificador do modelo"""
        try:
            encoding = tiktoken.encoding_for_model("deepseek-chat")
            return len(encoding.encode(text))
        except:
            # Fallback simples se não conseguir contar tokens
            return len(text.split()) // 3  # Aproximação grosseira

    def _truncate_documents(self, documents: List[Document], max_tokens: int = 60000) -> List[Document]:
        """Trunca documentos para ficar dentro do limite de tokens"""
        total_tokens = 0
        truncated_docs = []
        
        for doc in documents:
            doc_tokens = self._count_tokens(doc.page_content)
            if total_tokens + doc_tokens <= max_tokens:
                truncated_docs.append(doc)
                total_tokens += doc_tokens
            else:
                remaining_tokens = max_tokens - total_tokens
                if remaining_tokens > 1000:  # Só adiciona se valer a pena
                    # Trunca o conteúdo do documento
                    truncated_content = " ".join(doc.page_content.split()[:remaining_tokens*3])
                    truncated_doc = Document(
                        page_content=truncated_content,
                        metadata=doc.metadata
                    )
                    truncated_docs.append(truncated_doc)
                    break  # Já atingiu o limite
                    
        print(f"ℹ️ Documentos truncados para {total_tokens} tokens (limite: {max_tokens})")
        return truncated_docs
    
    def _setup_directories(self):
        """Cria diretórios necessários"""
        for directory in [self.docs_directory, self.cache_dir]:
            if not os.path.exists(directory):
                os.makedirs(directory)
                print(f"📁 Diretório '{directory}/' criado.")
    
    def _initialize_llm_openai(self):
        """Inicializa LLM e embeddings"""
        # Usando gpt-4o-mini do OpenAI
        print("🔧 Inicializando LLM OpenAI (gpt-4o-mini)...")
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        self.embeddings = OpenAIEmbeddings()

    def _initialize_llm_gemini(self):
        """Versão Gemini"""
        print("🔧 Inicializando LLM Google Gemini...")
        
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-exp",
            temperature=0,
            google_api_key=os.getenv("GOOGLE_API_KEY")
        )
                
        self.embeddings = OpenAIEmbeddings()


    def _initialize_llm_claude(self):
        """Versão Claude"""
        print("🔧 Inicializando LLM Anthropic Claude...")
        self.llm = ChatAnthropic(
            # model="claude-3-5-sonnet-20241022",
            model="claude-3-7-sonnet-latest",
            temperature=0,
            anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"),
            max_tokens=1024,  # Aumentar limite de tokens
        )
        # Claude não tem embeddings, usar OpenAI como fallback
        self.embeddings = OpenAIEmbeddings()

    def _initialize_llm_groq_llama3(self):
        """Versão Groq (Ultra-rápido)"""        
        print("🔧 Inicializando LLM Groq (Llama 3.3)...")
        self.llm = ChatGroq(
            model="llama3-8b-8192",
            temperature=0,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        self.embeddings = OpenAIEmbeddings()

    def _initialize_llm_groq_gemma(self):
        """Versão Groq (Ultra-rápido)"""        
        print("🔧 Inicializando LLM Groq (Gemma 2.0)...")
        self.llm = ChatGroq(
            model="gemma-7b-it",
            temperature=0,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        self.embeddings = OpenAIEmbeddings()

    def _initialize_llm_groq_mistral(self):
        """Versão Groq (Ultra-rápido)"""        
        print("🔧 Inicializando LLM Groq (Mixtral 8x7B)..." )
        self.llm = ChatGroq(
            model="mixtral-8x7b-32768",
            temperature=0,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        self.embeddings = OpenAIEmbeddings()


    def _initialize_llm_deepseek(self):
        """Versão DeepSeek (OpenAI API compatível)"""
        print("🔧 Inicializando LLM DeepSeek (DeepSeek Chat)...")
        self.llm = ChatOpenAI(
            model="deepseek-chat",
            temperature=0,
            openai_api_key=os.getenv("DEEPSEEK_API_KEY"),
            openai_api_base="https://api.deepseek.com"
        )
        self.embeddings = OpenAIEmbeddings()
    
    # Exemplo de uso com seleção dinâmica
    def _initialize_llm_dynamic(self, provider: str):
        """Função dinâmica que escolhe o provider"""
        providers = {
            "openai": self._initialize_llm_openai,
            "gemini": self._initialize_llm_gemini,
            "claude": self._initialize_llm_claude,
            "groq_llama3": self._initialize_llm_groq_llama3,
            "groq_gemma": self._initialize_llm_groq_gemma,
            "groq_mistral": self._initialize_llm_groq_mistral,
            "deepseek": self._initialize_llm_deepseek
        }

        print(f"🔧 Inicializando LLM com : {provider}\n")
        print("="*60)
        
        if provider in providers:
            providers[provider]()
        else:
            available = ", ".join(providers.keys())
            raise ValueError(f"Provider '{provider}' não suportado. Disponíveis: {available}")

    
    def _ocr_pdf_page(self, pdf_doc, page_number):
        """Realiza OCR em uma página do PDF"""
        try:
            page = pdf_doc.load_page(page_number)
            pix = page.get_pixmap()
            img = Image.open(io.BytesIO(pix.tobytes("png")))
            text = pytesseract.image_to_string(img, lang='por')
            return text
        except Exception as e:
            print(f"    ❌ [OCR] Erro na página {page_number+1}: {e}")
            return ""
    
    def _load_documents_with_cache(self):
        """Carrega documentos com sistema de cache inteligente"""
        documentos_carregados = []
        print(f"📚 Carregando documentos de: {self.docs_directory}")
        
        for root, dirs, files in os.walk(self.docs_directory):
            for file_name in files:
                if file_name.lower().endswith(".pdf"):
                    caminho_arquivo_pdf = os.path.join(root, file_name)
                    cache_file_name = os.path.splitext(file_name)[0] + ".txt"
                    cache_file_path = os.path.join(self.cache_dir, cache_file_name)
                    
                    texto_completo = ""
                    
                    # Tentar carregar do cache
                    if os.path.exists(cache_file_path):
                        try:
                            with open(cache_file_path, "r", encoding="utf-8") as f:
                                texto_completo = f.read()
                            if texto_completo.strip():
                                documentos_carregados.append(
                                    Document(
                                        page_content=texto_completo, 
                                        metadata={"source": caminho_arquivo_pdf, "cached": True}
                                    )
                                )
                                # print(f"  ♻️  Cache: '{file_name}'")
                                continue
                        except Exception as e:
                            print(f"  ⚠️  Erro no cache para '{file_name}': {e}")
                    
                    # Processar PDF se não estiver no cache
                    try:
                        doc = fitz.open(caminho_arquivo_pdf)
                        print(f"  📄 Processando: '{file_name}'")
                        
                        for page_num, page in enumerate(doc):
                            page_text = page.get_text()
                            
                            # Se extração normal falhar, usar OCR
                            if not page_text.strip():
                                print(f"    🔍 OCR página {page_num + 1}")
                                ocr_text = self._ocr_pdf_page(doc, page_num)
                                if ocr_text.strip():
                                    page_text = ocr_text
                            
                            if page_text.strip():
                                texto_completo += page_text + "\n\n"
                        
                        if texto_completo.strip():
                            documentos_carregados.append(
                                Document(
                                    page_content=texto_completo, 
                                    metadata={"source": caminho_arquivo_pdf, "cached": False}
                                )
                            )
                            
                            # Salvar no cache
                            with open(cache_file_path, "w", encoding="utf-8") as f:
                                f.write(texto_completo)
                            print(f"    💾 Salvo no cache")
                        
                        doc.close()
                    except Exception as e:
                        print(f"  ❌ Erro ao processar '{file_name}': {e}")
        
        print(f"✅ Total: {len(documentos_carregados)} documentos carregados")
        return documentos_carregados
    
    # --- NÓNS DO GRAFO ---
    def _load_documents_node(self, state: AgentState) -> AgentState:
        """Nó para carregar documentos"""
        if not os.path.exists(self.docs_directory) or not os.listdir(self.docs_directory):
            return {
                "retriever_initialized": False, 
                "answer": f"❌ Diretório '{self.docs_directory}' vazio ou inexistente."
            }
        
        docs = self._load_documents_with_cache()
        
        if not docs:
            return {
                "retriever_initialized": False, 
                "answer": "❌ Nenhum documento válido encontrado."
            }
        
        # Dividir documentos
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500, 
            chunk_overlap=100
        )
        split_docs = text_splitter.split_documents(docs)
        
        # Criar retriever
        vectorstore = Chroma.from_documents(
            documents=split_docs, 
            embedding=self.embeddings
        )
        retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
        
        # Atualizar sessão
        self.session.retriever = retriever
        self.session.retriever_initialized = True
        self.session.documents_loaded = True
        
        return {
            "documents": split_docs,
            "retriever": retriever,
            "retriever_initialized": True,
            "answer": ""
        }
    
    def _retrieve_documents_node(self, state: AgentState) -> AgentState:
        """Nó para buscar documentos relevantes"""
        question = state["question"]
        
        # Usar retriever da sessão se disponível
        retriever = self.session.retriever if self.session.retriever_initialized else state.get("retriever")
        
        if retriever is None:
            return {
                "documents": [], 
                "answer": "❌ Retriever não configurado."
            }
        
        documents_for_qa = retriever.invoke(question)
        return {"documents": documents_for_qa}

    def _filter_relevant_documents(self, documents: List[Document], question: str) -> List[Document]:
        """Filtra documentos por relevância usando embeddings"""
        question_embedding = self.embeddings.embed_query(question)
        scored_docs = []
        
        for doc in documents:
            doc_embedding = self.embeddings.embed_query(doc.page_content)
            similarity = np.dot(question_embedding, doc_embedding)
            scored_docs.append((similarity, doc))
        
        # Ordena por similaridade e pega os mais relevantes
        scored_docs.sort(reverse=True, key=lambda x: x[0])
        return [doc for _, doc in scored_docs[:3]]  # Apenas os 3 mais relevantes
    
    def _generate_answer_node(self, state: AgentState) -> AgentState:
        """Nó para gerar resposta com todas as otimizações"""
        question = state["question"]
        documents = state["documents"]
        
        if not documents:
            return {"answer": "❌ Nenhum documento relevante encontrado."}
        
        # Filtra documentos por relevância
        relevant_docs = self._filter_relevant_documents(documents, question)
        
        # Trunca documentos para caber no limite do modelo
        truncated_docs = self._truncate_documents(relevant_docs, max_tokens=60000)
        
        # Prompt especializado para condomínio
        prompt = ChatPromptTemplate.from_template("""
        Você é um assistente especializado em Q&A para o Condomínio Solar Trindade.
        Use os documentos fornecidos (atas, contratos, comunicados, regulamentos) para responder à pergunta.
        
        INSTRUÇÕES:
        - Seja preciso e específico nas datas e informações
        - Se não souber, diga "Não tenho informações suficientes nos documentos"
        - Para listas, use formatação clara com numeração
        - Mantenha tom profissional mas acessível
        
        Contexto dos documentos: {context}
        
        Pergunta: {input}
        
        Resposta:
        """)
        
        document_chain = create_stuff_documents_chain(self.llm, prompt)
        try:
            response = document_chain.invoke({
                "input": question, 
                "context": truncated_docs
            })
            return {"answer": response}
        except Exception as e:
            return {"answer": f"❌ Erro ao gerar resposta: {str(e)}"}
        
    
    def _decide_next_step(self, state: AgentState) -> str:
        """Decide próximo passo baseado no estado"""
        # Verificar se retriever está inicializado na sessão ou no estado
        if self.session.retriever_initialized or state.get("retriever_initialized"):
            return "retrieve_docs"
        else:
            return "load_docs"
    
    def _build_graph(self):
        """Constrói o grafo LangGraph"""
        workflow = StateGraph(AgentState)
        
        # Adicionar nós
        workflow.add_node("decide_initial_step", lambda state: state)
        workflow.add_node("load_docs", self._load_documents_node)
        workflow.add_node("retrieve_docs", self._retrieve_documents_node)
        workflow.add_node("generate_answer", self._generate_answer_node)
        
        # Configurar fluxo
        workflow.set_entry_point("decide_initial_step")
        
        workflow.add_conditional_edges(
            "decide_initial_step",
            self._decide_next_step,
            {
                "load_docs": "load_docs",
                "retrieve_docs": "retrieve_docs"
            }
        )
        
        workflow.add_edge("load_docs", "retrieve_docs")
        workflow.add_edge("retrieve_docs", "generate_answer")
        workflow.add_edge("generate_answer", END)
        
        self.app = workflow.compile()
    
    # --- INTERFACE PÚBLICA ---
    def ask_question(self, question: str, show_process: bool = False) -> str:
        """Faz uma pergunta ao agente"""
        print(f"❓ {question}")
        
        if show_process:
            print("🔄 Processando...")
        
        # Preparar estado inicial
        initial_state = {
            "question": question,
            "documents": [],
            "answer": "",
            "retriever": self.session.retriever,
            "retriever_initialized": self.session.retriever_initialized,
            "conversation_history": self.session.conversation_history,
            "session_context": self.session
        }
        
        # Executar grafo
        current_result = None
        for step in self.app.stream(initial_state):
            if show_process:
                node_name = list(step.keys())[0] if step else "unknown"
                print(f"  🔸 {node_name}")
            current_result = step
        
        # Processar resultado
        if current_result:
            final_state = list(current_result.values())[0]
            answer = final_state.get("answer", "❌ Erro ao processar pergunta")
            
            # Atualizar histórico
            qa_pair = {
                "question": question,
                "answer": answer,
                "timestamp": datetime.now().isoformat()
            }
            self.session.conversation_history.append(qa_pair)
            
            return answer
        
        return "❌ Erro no processamento"
    
    def ask_and_display(self, question: str, show_process: bool = False):
        """Faz pergunta e exibe resposta formatada"""
        answer = self.ask_question(question, show_process)
        print(f"\n💡 **Resposta:**")
        display(Markdown(answer))
        return answer
    
    def show_conversation_history(self, limit: int = 5):
        """Mostra histórico de conversas"""
        print(f"\n📋 **Histórico de Conversas** (últimas {limit}):")
        print("-" * 60)
        
        recent = self.session.conversation_history[-limit:]
        
        for i, qa in enumerate(recent, 1):
            timestamp = qa.get("timestamp", "N/A")
            question = qa.get("question", "")
            answer = qa.get("answer", "")
            
            print(f"\n**{i}.** 🕒 {timestamp}")
            print(f"❓ {question}")
            print(f"💡 {answer[:100]}{'...' if len(answer) > 100 else ''}")
    
    def get_session_info(self):
        """Informações da sessão"""
        return {
            "session_id": self.session.session_id,
            "total_questions": len(self.session.conversation_history),
            "retriever_initialized": self.session.retriever_initialized,
            "documents_loaded": self.session.documents_loaded,
            "docs_directory": self.docs_directory,
            "cache_directory": self.cache_dir
        }

# --- EXECUÇÃO PRINCIPAL ---
def run_solar_qa_demo(provider: str = "openai"):
    """Executa demonstração do agente Solar Q&A"""
    
    print("🏢 === AGENTE Q&A CONDOMÍNIO SOLAR TRINDADE ===")
    print("="*60)
    
    # Inicializar agente
    qa_agent = SolarCondominiumQA(provider=provider)
    
    # Perguntas de exemplo
    sample_questions = [
        "Qual a data da última reunião ou assembleia do condomínio?",
        #"Quais as datas das cinco últimas reuniões ou assembleias?",
        "Quais foram os principais assuntos da última assembleia?",
        "quais os nomes dos membros do conselho fiscal atuais?",
        #"Como base na ultima assembleia, qual o valor da taxa condominial atual?"
    ]
    
    print(f"📊 **Informações da Sessão:**")
    for key, value in qa_agent.get_session_info().items():
        print(f"  • {key}: {value}")
    
    print("\n" + "="*60)
    
    # Executar perguntas
    for i, question in enumerate(sample_questions, 1):
        print(f"\n🔸 **Pergunta {i}/{len(sample_questions)}**")
        qa_agent.ask_and_display(question, show_process=(i == 1))
        
        if i < len(sample_questions):
            print("\n" + "-"*40)
    
    # Mostrar histórico
    qa_agent.show_conversation_history()
    
    print(f"\n✅ **Sessão Finalizada:** {qa_agent.session.session_id}")
    print(f"📈 **Total de Perguntas:** {len(qa_agent.session.conversation_history)}")
    
    return qa_agent

def interactive_solar_qa():
    """Modo interativo para o Solar Q&A"""
    qa_agent = SolarCondominiumQA()
    
    print("🏢 === MODO INTERATIVO - SOLAR TRINDADE ===")
    print("Comandos: 'sair', 'historico', 'info', 'limpar'")
    print("="*60)
    
    while True:
        try:
            question = input("\n❓ Sua pergunta: ").strip()
            
            if question.lower() in ['sair', 'exit', 'quit']:
                print("👋 Encerrando...")
                break
            elif question.lower() in ['historico', 'history']:
                qa_agent.show_conversation_history()
                continue
            elif question.lower() == 'info':
                print("\n📊 **Informações da Sessão:**")
                for key, value in qa_agent.get_session_info().items():
                    print(f"  • {key}: {value}")
                continue
            elif question.lower() in ['limpar', 'clear']:
                qa_agent.session.conversation_history = []
                print("🗑️ Histórico limpo")
                continue
            elif not question:
                continue
            
            qa_agent.ask_and_display(question)
            
        except KeyboardInterrupt:
            print("\n👋 Interrompido pelo usuário")
            break
        except Exception as e:
            print(f"❌ Erro: {e}")
    
    return qa_agent


In [2]:

# --- EXECUÇÃO ---
if __name__ == "__main__":
    # Escolha o modo de execução:
    # qa_agent = run_solar_qa_demo("claude")     # Demo automática
    # qa_agent = interactive_solar_qa()   # Modo interativo

    providers = [
        "openai", 
        "gemini", 
        # "claude",
        # "groq_llama3", 
        # "groq_gemma", 
        # "groq_mistral", 
        # "deepseek"
    ]

    for provider in providers:
        print(f"\n🔧 Inicializando LLM: {provider}")
        run_solar_qa_demo(provider)


🔧 Inicializando LLM: openai
🏢 === AGENTE Q&A CONDOMÍNIO SOLAR TRINDADE ===
🔧 Inicializando LLM com : openai

🔧 Inicializando LLM OpenAI (gpt-4o-mini)...
📊 **Informações da Sessão:**
  • session_id: session_20250620_191404
  • total_questions: 0
  • retriever_initialized: False
  • documents_loaded: False
  • docs_directory: docs_condominio
  • cache_directory: processed_docs_cache


🔸 **Pergunta 1/3**
❓ Qual a data da última reunião ou assembleia do condomínio?
🔄 Processando...
  🔸 decide_initial_step
📚 Carregando documentos de: docs_condominio
✅ Total: 31 documentos carregados
  🔸 load_docs
  🔸 retrieve_docs
ℹ️ Documentos truncados para 71 tokens (limite: 60000)
  🔸 generate_answer

💡 **Resposta:**


A última reunião do Condomínio Solar Trindade ocorreu no dia 6 de agosto de 2024, em segunda convocação, às 20h, no salão de festas do condomínio.


----------------------------------------

🔸 **Pergunta 2/3**
❓ Quais foram os principais assuntos da última assembleia?
ℹ️ Documentos truncados para 49 tokens (limite: 60000)

💡 **Resposta:**


Na última assembleia, realizada em 04 de novembro de 2024, os principais assuntos abordados foram:

1. **Atualização do Regimento Interno**: Foi discutido que não foi possível encaminhar a votação para a aprovação da atualização do Regimento Interno devido a pendências. Decidiu-se agendar uma nova assembleia para tratar desse assunto.

2. **Previsão Orçamentária para 2024**: A Sra. Maria Fernanda, representante da Controlar, apresentou e explicou item a item da minuta da previsão orçamentária para o ano de 2024, considerando o reajuste dos funcionários e a inflação.

3. **Itens Delicados**: A síndica mencionou que os itens 4, 5 e 6 da pauta eram delicados e que seria necessário dedicar mais tempo a esses assuntos, além do tempo médio de dez minutos previsto para a assembleia.

Esses foram os principais tópicos discutidos na assembleia.


----------------------------------------

🔸 **Pergunta 3/3**
❓ quais os nomes dos membros do conselho fiscal atuais?
ℹ️ Documentos truncados para 38 tokens (limite: 60000)

💡 **Resposta:**


...


----------------------------------------

🔸 **Pergunta 2/3**
❓ Quais foram os principais assuntos da última assembleia?
ℹ️ Documentos truncados para 42 tokens (limite: 60000)

💡 **Resposta:**


Na assembleia de 04 de novembro de 2024, o principal assunto foi a atualização do Regimento Interno do Condomínio Solar Trindade. Devido a pendências destacadas em sessões anteriores, não foi possível encaminhar a votação para aprovação ou não da atualização. Ficou decidido que uma nova assembleia será agendada para resolver as pendências e votar o documento final.


----------------------------------------

🔸 **Pergunta 3/3**
❓ quais os nomes dos membros do conselho fiscal atuais?
ℹ️ Documentos truncados para 38 tokens (limite: 60000)

💡 **Resposta:**


Não tenho informações suficientes nos documentos para fornecer os nomes dos membros do conselho fiscal atuais. Os documentos informam apenas sobre o processo de eleição e composição do Conselho Fiscal.


📋 **Histórico de Conversas** (últimas 5):
------------------------------------------------------------

**1.** 🕒 2025-06-20T19:31:05.556944
❓ Qual a data da última reunião ou assembleia do condomínio?
💡 A última Assembleia Geral Extraordinária do Condomínio do Edifício Solar Trindade, registrada nos do...

**2.** 🕒 2025-06-20T19:31:07.880871
❓ Quais foram os principais assuntos da última assembleia?
💡 Na assembleia de 04 de novembro de 2024, o principal assunto foi a atualização do Regimento Interno ...

**3.** 🕒 2025-06-20T19:31:10.161486
❓ quais os nomes dos membros do conselho fiscal atuais?
💡 Não tenho informações suficientes nos documentos para fornecer os nomes dos membros do conselho fisc...

✅ **Sessão Finalizada:** session_20250620_192239
📈 **Total de Perguntas:** 3


In [3]:
# qa_agent = SolarCondominiumQA()
# # resposta = qa_agent.ask_question("Qual a última assembleia? Resposta curta e resumida")
# resposta = qa_agent.ask_question("Quem são os membros do conselho fiscal? Resposta curta e resumida")
# qa_agent.show_conversation_history()